# HiForm Timber Harvest BMP Tool - identify and optionally export large patch timber harvests as polygons or points (04/19/2023)

(set the detailed stream layer to your State - go to line 1052, replace the 2-letter state abbrevition to your state, in lower case, keep formatting)
   (more detailed step-by-step instructions - https://drive.google.com/file/d/1kcesaTw2NSwX6r0k7_Wd3sbVWiNGe6kE/view?usp=share_link)

1. Pan and zoom the map display to your area of interest
   - using the Inspector tool tab (upper-right-panel), click the map's center, highlight the returned lat-long pair and right-click-copy (mentally denote the zoom level), paste to line 39, change the zoom level if different
2. Decide the extent from which to report, choose 1 of 3 possible:
   - by a single county (default), edit line 51, type the county name and state abbreviation (like shown), go to #3
  - by multiple counties, edit lines 67, 68 (as described above) at line 83, begin the line with double-forward slashes (//) at line 84, remove the double-forward slashes (//), go to #3
  - by a hand drawn polygon, using the maps "Geometry Imports" tool, select "new layer" to "Draw a shape", or 'rectangle', at lines 83 and 84 begin the lines with double-forward slashes (//) at line 85, remove the double-forward slashes (//), go to #33.
3. Use the default timeframe - use the default (2022) 1yr NDVI change product, go to #5
4. Change the timeframe of the pre- and post-disturbance dates
  - pre-disturbance date, line 94, choose to composite over a range of dates, or choose a single date of known cloud free imagery, go to #5
  - if a single date, in the first entry enter the exact date, for the second entry after the comma, advance the date by one day and enter the date in the secondary position
  - post-disturbance date, line 139, choose to composite over a range of dates, or choose a single date of known cloud free imagery, go to #5
  - if a single date, in the first entry enter the exact date, for the second entry after the comma, advance the date by one day and enter the date in the secondary position
5. Select "Run" at the top of the Code Editor window, and watch for the completion status in the "Layers" list of the map window (if needing to export a geospatial product, wait for the layers to draw before proceeding to #6)
6. Optional: to export, go to "Tasks" tab and select the "RUN" next to the export product of interest (save to Google-Drive)  

- Bill Christie (william.m.christie@usda.gov), Remote Sensing Analyst, Eastern Threat Center, Asheville, NC
- Steve Norman (steve.norman@usda.gov), Research Ecologist, Eastern Threat Center, Asheville, NC
- HiForm website - https://hiform.org;   HiForm-specific shared files - https://drive.google.com/drive/folders/1Z3-ZryyOsndq0iY5VME07XIJCtLPxvue?usp=sharing
- EFETAC website - https://forestthreats.org


In [3]:
import geemap
import ee
import math
ee.Authenticate()
ee.Initialize(project='ee-chrismihiar')

In [8]:
#  Center the map and set the zoom level

m = geemap.Map()
m.setCenter(-84.9122, 32.7199, 8)
m

Map(center=[32.7199, -84.9122], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

In [12]:
# Identify a specific county, or counties to map

counties = ee.FeatureCollection('TIGER/2018/Counties')
m.addLayer(counties, {}, 'counties (1:20M)', False)

# Select a single county

counties1 = ee.FeatureCollection('TIGER/2018/Counties') \
    .filter(ee.Filter.eq("NAME",'Harris, GA'))

geometry = counties1

empty = ee.Image().byte()
outline = empty.paint(**{
  'featureCollection': counties,
  'color': 1,
  'width': 2})

m.addLayer(outline, {'palette': '66ff00'}, 'county selected', False)

In [19]:
# Select multiple counties

filterCounties = ee.Filter.Or(
    ee.Filter.eq("NAME",'Harris, GA'),
    ee.Filter.eq("NAME",'Talbot, GA'))

counties2 = ee.FeatureCollection('TIGER/2018/Counties') \
.filter(filterCounties)

empty = ee.Image().byte()
outline2 = empty.paint(**{
  'featureCollection': counties2,
  'color': 1,
  'width': 2})


m.addLayer(outline2,{'palette': '66ff00'}, 'counties selected', False) #TRUE would toggle this layer on
m

geometry = counties  # comment this line out when you draw your own polygon (geometry)
# geometry = counties2;  # comment this line out when you draw your own polygon (geometry)
# geometry = geometry2;  # un-comment this line when you draw your own polygon (geometry)


In [21]:
################################
###  Load a Sentinel2 pre-disturbance image
################################

#pre = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')\
#.filterDate('2021-05-21', '2021-07-21').filterBounds(geometry)

pre = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
.filterDate('2020-07-21', '2020-09-21') #  1yr growing season baseline, historical \

#print(pre,"pre")

# Add NDVI and DATE as separate bands to image layer stack
def addNDVI(pre):
  ndvi = pre.normalizedDifference(['B8', 'B4']).rename('NDVI')
  return pre.addBands(ndvi)

def addDate(pre):
  date = ee.Date(pre.get('system:time_start'))
  dateString = date.format('yyyyMMdd')
  dateNumber = ee.Number.parse(dateString)
  dateBand = ee.Image.constant(dateNumber).uint32().rename('date')
  return pre.addBands(dateBand)

pre2 = pre.map(addNDVI).map(addDate)

# "greenest" is for max ndvi value compositing of multiple dates
greenest_pre2 = pre2.qualityMosaic('NDVI')

# Perform clip on larger extent 'bounds' image to geometry or shapefile
greenest_pre2_clip = greenest_pre2.clip(geometry)

# Display the 'pre' date raster with random colors
m.addLayer(greenest_pre2_clip.select('date').randomVisualizer(), {}, 'pre date raster', True)

#  Display pre products
#m.addLayer(greenest_pre2_clip.select(['B8', 'B4', 'B3']), {min: 900, max: 3000}, 'pre CIR', False)
#m.addLayer(greenest_pre2_clip.select(['B4', 'B3', 'B2']), {min: 300, max: 1900, gamma: 1.0}, 'pre natural color TOA', False)
m.addLayer(greenest_pre2_clip.select(['B4', 'B3', 'B2']), {'min': 50, 'max': 1200, 'gamma': 1.5}, 'pre natural color composite', False)
m
#Export.image.toDrive(**{image: greenest_pre2_clip.select(['B2', 'B3', 'B4', 'B8']),description: 'S2_GA_Tornado_pre022419_B2348',region:geometry,'scale':10, 'maxPixels':1e13})


Map(bottom=7114.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Searc…

In [ ]:
############################/
# Load a Sentinel2 post-disturbance image
############################/

post = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')\
.filterDate('2022-07-21', '2022-09-21').filterBounds(geometry)

#post = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \

# .filterDate('2022-05-01', '2022-06-21') #  within growing season baseline, display the pre natural color to look for cloud contamination \


#print(post,"post")

# Add NDVI and DATE as separate bands to image layer stack
def addNDVI(post):
  ndvi = post.normalizedDifference(['B8', 'B4']).rename('NDVI')
  return post.addBands(ndvi)

def addDate(img):
  date = ee.Date(img.get('system:time_start'))
  dateString = date.format('yyyyMMdd')
  dateNumber = ee.Number.parse(dateString)
  dateBand = ee.Image.constant(dateNumber).uint32().rename('date')
  return img.addBands(dateBand)

post2 = post.map(addNDVI).map(addDate)

# Make a "greenest" pixel composite.
greenest_post2 = post2.qualityMosaic('NDVI')

#print (greenest_post2, 'greenest_post2')

# Perform clip on larger extent 'bounds' image to geometry or shapefile
greenest_post2_clip = greenest_post2.clip(geometry)

# Display the 'post' date raster with random colors
m.addLayer(greenest_post2_clip.select('date').randomVisualizer(), {}, 'post date raster', False)

#  Display post products
#m.addLayer(greenest_post2_clip.select(['B8', 'B4', 'B3']), {min: 341, max: 3649}, 'post CIR', False)
#m.addLayer(greenest_post2_clip.select(['B4', 'B3', 'B2']), {min: 300, max: 1900, gamma: 1}, 'post nat color TOA', False)
m.addLayer(greenest_post2_clip.select(['B4', 'B3', 'B2']), {'min': 0, 'max': 1600, 'gamma': 1.5}, 'post natural color composite', False)

# ####/ if you want to download the post image, bands 2348 let you create natural color, CIR and NDVI values
# Export.image.toDrive(**{image: greenest_post2_clip.select(['B2', 'B3', 'B4', 'B8']),description: 'S2sr_cusp_b2348_post091522_v103122','region':geometry,'scale':10, 'maxPixels':1e13})

In [ ]:
# ############################/
# # Calculate Absolute NDVI Change
# ############################

# #    Perform absolute change NDVI image differencing
postMINUSpre = (greenest_post2_clip.subtract(greenest_pre2_clip))

####   ABS NDVI CHANGE rescale to signed integer 8bit (-127 - 127) to reduce file size
absNDVIc = (postMINUSpre.select('NDVI'))
#print (pcNDVI,'pcNDVI')

absNDVIc_x100 = absNDVIc.multiply(100)

#print(absNDVIc_x100,'absNDVIc_x100')

absNDVIc_x100sint8 = absNDVIc_x100.int8()
m.addLayer(absNDVIc_x100sint8, {min: -38, max:19}, 'absNDVIc_x100sint8', False)

#print(absNDVIc_x100sint8,'absNDVIc_x100sint8')

In [42]:
###########################/
###/  EXPORT ALL-LANDS, RESCALED, FULL DATA RANGE ABSOLUTE NDVI CHANGE RASTER - 8BIT SIGNED INTEGER, VALUES -127 - 127
###/  ESRI COLOR MAP AND LEGEND - https:#drive.google.com/drive/folders/1XAsaPDagRNNh1-T1LvZQqO6l2BEb19CA?usp=sharing
###########################/

#Export.image.toDrive(**{image: absNDVIc_x100sint8, description: 'S2TOA_TN_DecaturCo_Hail_030319_dNDVI_sint8bit_pre051819_post060120_v072320',region:geometry,'scale':10, 'maxPixels':1e13})
#    filename suggestion -                               sensor-state-event-evdate-product-format-preimage-postimage-versiondate

######/ 10m elevation and slope
dataset = ee.Image('USGS/3DEP/10m')
elevation = dataset.select('elevation')
slope = ee.Terrain.slope(elevation)
# m.setCenter(-84.8996, 32.60217, 12)
# m.addLayer(elevation.clip(geometry), {min: 0, max: 3000,   palette: [
#     '3ae237', 'b5e22e', 'd6e21f', 'fff705', 'ffd611', 'ffb613', 'ff8b13',
#     'ff6e08', 'ff500d', 'ff0000', 'de0101', 'c21301', '0602ff', '235cb1',
#     '307ef3', '269db1', '30c8e2', '32d3ef', '3be285', '3ff38f', '86e26f'
#   ],
# }, 'elevation', False)

percent_slope = slope.divide(180).multiply(math.pi).tan().multiply(100).rename('percent').round()

m.addLayer(percent_slope.clip(geometry), {}, 'percent slope', False)

#########/  floodplain forest

fp = ee.Image('users/timberharvestmap/floodplains_eastern_epaeast')
m.addLayer(fp.clip(geometry), {'palette': ['0700d6']}, 'floodplains (bottomland hardwoods)', False)

In [ ]:

###################################################
###  ASSIGN PRIOR GROWING SEASON PERIOD FOR SUMMER MAX COMPOSITING
###################################################

gsmax = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')\
  .filterDate('2021-05-01', '2021-09-21')\
  .filterBounds(geometry)
#gsmax = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
#gsavg = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') # use average/mean on single dates \
#.filterDate('2020-06-01', '2020-09-21') \

#print(gsmax,"gsmax")
#print(gsavg,"gsavg")

# Add NDVI and DATE as separate bands to image layer stack(s)

def addNDVI(gsmax):
  ndvi = gsmax.normalizedDifference(['B8', 'B4']).rename('NDVI')
  return gsmax.addBands(ndvi)

# def addNDVI(gsavg):
#   ndvi = gsavg.normalizedDifference(['B8', 'B4']).rename('NDVI')
#   return gsavg.addBands(ndvi)

def addDate(img):
  date = ee.Date(img.get('system:time_start'))
  dateString = date.format('yyyyMMdd')
  dateNumber = ee.Number.parse(dateString)
  dateBand = ee.Image.constant(dateNumber).uint32().rename('date')
  return img.addBands(dateBand)

withNDVIDATE_gsmax = gsmax.map(addNDVI).map(addDate)
#withNDVIDATE_gsavg = gsavg.map(addNDVI).map(addDate)

# Lists 'collection return' to console with NDVI and DATE band added
#print (withNDVIDATE_gsmax, 'withNDVIDATE_gsmax')
#print (withNDVIDATE_gsavg, 'withNDVIDATE_gsavg')

# "greenest" is for max ndvi value compositing of multiple dates
greenest_withNDVIDATE_gsmax = withNDVIDATE_gsmax.qualityMosaic('NDVI')

# greenest_withNDVIDATE_gsavg = withNDVIDATE_gsavg.mean()
# print (greenest_withNDVIDATE_gsavg, 'greenest_withNDVIDATE_gsavg')
# print (greenest_withNDVI_gsmax_winter, 'greenest_withNDVI_gsmax_winter')

#m.addLayer(greenest_withNDVIDATE_gsmax_winter.select(['B8', 'B4', 'B3']), {min: 341, max: 3649}, 'gsmax CIR gsmax-clip')

# Perform clip on larger extent 'bounds' image to geometry or shapefile
greenest_withNDVIDATE_gsmax_clip = greenest_withNDVIDATE_gsmax.clip(geometry)

#m.addLayer(greenest_withNDVIDATE_gsavg_clip.select(['B8', 'B4', 'B3']), {}, 'cir gsavg ndvi mean clip')

# #  Display the the date raster
# #m.addLayer(greenest_withNDVIDATE_gsmax_winter_clip.select('date').randomVisualizer(), {}, 'gsmax date raster random colors')
# #m.addLayer(withNDVI_gsmax_winter_clip.select(['NDVI']), {min: -0.57, max: 0.78}, 'withNDVIDATE_gsmax_winter_clip NDVI b&w')
m.addLayer(greenest_withNDVIDATE_gsmax_clip.select(['B4', 'B3', 'B2']), {'min': 100, 'max': 1500}, 'gsmax TCC')
m

# #Export.image.toDrive(**{image: greenest_withNDVIDATE_gsmax_winter_clip.select(['B2', 'B3', 'B4', 'B8']),description: 'name',region:geometry,'scale':10, 'maxPixels':1e13})


In [44]:
#####################################################
###   ASSIGN PRIOR WINTER SEASON PERIOD FOR THE LEAF-OFF SEASON MAX
####################################################/

winmax = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')\
  .filterDate('2021-01-01', '2021-03-15')\
  .filterBounds(geometry)
#winmax = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
#.filterDate('2020-01-01', '2020-03-01') \


#print(winmax,"winmax")

### Add NDVI as a band named NDVI to image layer stacks
def addNDVI(winmax):
  ndvi = winmax.normalizedDifference(['B8', 'B4']).rename('NDVI')
  return winmax.addBands(ndvi)

def addDate(img):
  date = ee.Date(img.get('system:time_start'))
  dateString = date.format('yyyyMMdd')
  dateNumber = ee.Number.parse(dateString)
  dateBand = ee.Image.constant(dateNumber).uint32().rename('date')
  return img.addBands(dateBand)

withNDVIDATE_winmax = winmax.map(addNDVI).map(addDate)

# # Lists 'collection return' to console with NDVI layer added to band stack
#print (withNDVIDATE_winmax, 'withNDVIDATE_winmax')

# Make a "greenest" pixel composite.
greenest_withNDVIDATE_winmax = withNDVIDATE_winmax.qualityMosaic('NDVI')
#greenest_withNDVIDATE_winavg = withNDVIDATE_winavg.mean()

# print (greenest_withNDVI_winmax, 'withNDVI_winmax')

# Perform clip on larger extent 'bounds' image to geometry or shapefile
greenest_withNDVIDATE_winmax_clip = greenest_withNDVIDATE_winmax.clip(geometry)

# Display the date raster with random colors.
#m.addLayer(greenest_withNDVIDATE_winmax_winter_clip.select('date').randomVisualizer(), {}, 'winmax date')

In [74]:
# ###################################################/
# # CALCULATE ABSOLUTE NDVI RANGE - SUMMER TO WINTER
# ###################################################/

# Perform absolute change NDVI image differencing
gsmaxMINUSwinmax = (greenest_withNDVIDATE_gsmax_clip.subtract(greenest_withNDVIDATE_winmax_clip))
#print(gsmaxMINUSwinmax,'gsmaxMINUSwinmax')

ADD_gsmaxMINUSwinmax = (greenest_withNDVIDATE_gsmax_clip.add(greenest_withNDVIDATE_winmax_clip))
#print(ADD_gsmaxMINUSwinmax,'ADD_gsmaxMINUSwinmax')

mean_ST = ADD_gsmaxMINUSwinmax.multiply(0.5)
#print(mean_ST,'mean_ST')

#rescale NDVI

mean_ST2 = (mean_ST.select('NDVI'))
mean_ST2_x100 = mean_ST2.multiply(100)
mean_ST2_x100sint8 = mean_ST2_x100.int8()
#print(mean_ST2_x100sint8,'mean_ST2_x100sint8')

##m.addLayer(mean_ST2_x100sint8.select('NDVI'), imageVisParam3, 'mean_ST2_x100sint8')
#m.addLayer(mean_ST.select(['B4', 'B3', 'B2']), {min: 360, max: 2160, gamma: 1.5}, 'meanST nat color', False)
#m.addLayer(mean_ST2_x100sint8.select('NDVI'), {}, 'find bin break mean_ST2_x100sint8', False)

mean_ST2_x100sint8_binned = mean_ST2_x100sint8.select(['NDVI']).gte(45)
#m.addLayer(mean_ST2_x100sint8_binned, {}, 'binned break', False)

mean_ST2_x100sint8_binnedMASK = mean_ST2_x100sint8_binned.mask(mean_ST2_x100sint8_binned)
# print(mean_ST2_x100sint8_binnedMASK,'mean_ST2_x100sint8_binnedMASK')
# m.addLayer(mean_ST2_x100sint8_binnedMASK, {}, 'mean_ST2_x100sint8_binnedMASK', False)

###   rescale to signed integer 8bit (-127 - 127) to reduce file size
range = (gsmaxMINUSwinmax.select('NDVI'))

range_x100 = range.multiply(100)
#print(range_x100,'range_x100')

range_x100sint8 = range_x100.int8()
#print(range_x100sint8,'range_x100sint8')

m.addLayer(range_x100sint8, {min: -7, max: 58}, 'range_x100sint8')
m


Map(bottom=3389166.0027878345, center=[32.64161531546355, -84.83651314082796], controls=(WidgetControl(options…

In [73]:
# ###############################
# ##  Seasonal Threshold annual NDVI range to create DECIDUOUS mask
# ###############################

dec_range_x100sint8 = range_x100sint8.select(['NDVI']).gte(21)

m.addLayer(dec_range_x100sint8, {}, 'dec-forest from range')
m

# ###############################
# ##  Seasonal Threshold annual NDVI range to create EVERGREEN-MIXED forest mask
# ###############################

# evgrn_range_x100sint8 = range_x100sint8.select(['NDVI']).lte(20)
# evgrn_range_x100sint8 = range_x100sint8.select(['NDVI']).gte(-9).And(range_x100sint8.select(['NDVI']).lte(7))

# #m.addLayer(evgrn_range_x100sint8, {}, 'evgrn-forest from range',False)
# #m.addLayer(range_x100sint8.mask(evgrn_range_x100sint8), imageVisParam2, 'eg annual ndvi range colorized')

# #m.addLayer(greenest_pre4_clip.select(['B4', 'B3', 'B2']).mask(evgrn_range_x100sint8), {min: 100, max: 1000, gamma: 1.5}, 'pre4 winter pine nat color', False)

Map(bottom=26587.000576793314, center=[33.50933672598489, -84.2097502128431], controls=(WidgetControl(options=…

In [72]:
#######################/
# Load NAIP aerial photography in b/w or color - start with designating NAIP year 2019, no results, move to 2018, etc.
#
#   this is used to (1) produce a visualization where transparent change is overlayed on a version of b/w naip
#                   (2) display a known, or more current color naip aerial photography than default imager served in EE
#
########################/

# #  N A I P

# # https:#developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ?hl=en
# # Dataset Availability - 2002-06-15T00:00:00Z–2020-12-17T00:00:00

dataset = ee.ImageCollection('USDA/NAIP/DOQQ')\
                  .filter(ee.Filter.date('2019-01-01', '2019-12-31'))\
                  .filterBounds(geometry)

# print(dataset.aggregate_array('system:index').getInfo(),'dataset')

TrueColor = dataset.select(['R', 'G', 'B'])
# # # FalseColor = dataset.select(['N', 'R', 'G'])
# # # bwRed = dataset.select(['R'])
# # # bwNIR = dataset.select(['N'])

TrueColorVis = {'min': 50, 'max': 255, 'gamma': 2}

# # print(bwRed,'b/w NAIP')

# # m.addLayer(bwRed, imageVisParam, 'NAIP B/W from Red Band')
m = geemap.Map()
m.setCenter(-84.9122, 32.7199, 8)
m.addLayer(TrueColor, TrueColorVis, '2019 NAIP True Color')
m
# #m.addLayer(FalseColor, TrueColorVis, 'NAIP False Color', False)
# #m.addLayer(bwNIR, {}, 'NAIP B/W from NIR Band')

# # NAIP dataset description - https:#developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ#description

Map(center=[32.7199, -84.9122], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

In [48]:
#########################
# Below is used to convert the signed int 8-bit DN raster to a light-weight, photo-like RGB composite, for display or export, cannot modify values on desktop
#########################

sld_intervals = (
'<RasterSymbolizer>' + \
'<ColorMap type="intervals" extended="False" >' + \
'<ColorMapEntry color="#000096" quantity="127" />' + \
'<ColorMapEntry color="#000096" quantity="126" />' + \
'<ColorMapEntry color="#000096" quantity="125" />' + \
'<ColorMapEntry color="#000096" quantity="124" />' + \
'<ColorMapEntry color="#000096" quantity="123" />' + \
'<ColorMapEntry color="#000096" quantity="122" />' + \
'<ColorMapEntry color="#000096" quantity="121" />' + \
'<ColorMapEntry color="#000096" quantity="120" />' + \
'<ColorMapEntry color="#000096" quantity="119" />' + \
'<ColorMapEntry color="#000096" quantity="118" />' + \
'<ColorMapEntry color="#000096" quantity="117" />' + \
'<ColorMapEntry color="#000096" quantity="116" />' + \
'<ColorMapEntry color="#000096" quantity="115" />' + \
'<ColorMapEntry color="#000096" quantity="114" />' + \
'<ColorMapEntry color="#000096" quantity="113" />' + \
'<ColorMapEntry color="#000096" quantity="112" />' + \
'<ColorMapEntry color="#000096" quantity="111" />' + \
'<ColorMapEntry color="#000096" quantity="110" />' + \
'<ColorMapEntry color="#000096" quantity="109" />' + \
'<ColorMapEntry color="#000096" quantity="108" />' + \
'<ColorMapEntry color="#000096" quantity="107" />' + \
'<ColorMapEntry color="#000096" quantity="106" />' + \
'<ColorMapEntry color="#000096" quantity="105" />' + \
'<ColorMapEntry color="#000096" quantity="104" />' + \
'<ColorMapEntry color="#000096" quantity="103" />' + \
'<ColorMapEntry color="#000096" quantity="102" />' + \
'<ColorMapEntry color="#000096" quantity="101" />' + \
'<ColorMapEntry color="#000096" quantity="100" />' + \
'<ColorMapEntry color="#000096" quantity="99" />' + \
'<ColorMapEntry color="#000096" quantity="98" />' + \
'<ColorMapEntry color="#000096" quantity="97" />' + \
'<ColorMapEntry color="#000096" quantity="96" />' + \
'<ColorMapEntry color="#000096" quantity="95" />' + \
'<ColorMapEntry color="#000096" quantity="94" />' + \
'<ColorMapEntry color="#000096" quantity="93" />' + \
'<ColorMapEntry color="#000096" quantity="99" />' + \
'<ColorMapEntry color="#000096" quantity="98" />' + \
'<ColorMapEntry color="#000096" quantity="97" />' + \
'<ColorMapEntry color="#000096" quantity="96" />' + \
'<ColorMapEntry color="#000096" quantity="95" />' + \
'<ColorMapEntry color="#000096" quantity="94" />' + \
'<ColorMapEntry color="#000096" quantity="93" />' + \
'<ColorMapEntry color="#000096" quantity="92" />' + \
'<ColorMapEntry color="#000096" quantity="91" />' + \
'<ColorMapEntry color="#000096" quantity="90" />' + \
'<ColorMapEntry color="#000096" quantity="89" />' + \
'<ColorMapEntry color="#000096" quantity="88" />' + \
'<ColorMapEntry color="#000096" quantity="87" />' + \
'<ColorMapEntry color="#000096" quantity="86" />' + \
'<ColorMapEntry color="#000096" quantity="85" />' + \
'<ColorMapEntry color="#000096" quantity="84" />' + \
'<ColorMapEntry color="#000096" quantity="83" />' + \
'<ColorMapEntry color="#000096" quantity="82" />' + \
'<ColorMapEntry color="#000096" quantity="81" />' + \
'<ColorMapEntry color="#000096" quantity="80" />' + \
'<ColorMapEntry color="#000096" quantity="79" />' + \
'<ColorMapEntry color="#000096" quantity="78" />' + \
'<ColorMapEntry color="#000096" quantity="77" />' + \
'<ColorMapEntry color="#000096" quantity="76" />' + \
'<ColorMapEntry color="#000096" quantity="75" />' + \
'<ColorMapEntry color="#000096" quantity="73" />' + \
'<ColorMapEntry color="#000096" quantity="72" />' + \
'<ColorMapEntry color="#000096" quantity="71" />' + \
'<ColorMapEntry color="#000096" quantity="70" />' + \
'<ColorMapEntry color="#000096" quantity="69" />' + \
'<ColorMapEntry color="#000096" quantity="68" />' + \
'<ColorMapEntry color="#000096" quantity="67" />' + \
'<ColorMapEntry color="#000096" quantity="66" />' + \
'<ColorMapEntry color="#000096" quantity="65" />' + \
'<ColorMapEntry color="#000096" quantity="64" />' + \
'<ColorMapEntry color="#000096" quantity="63" />' + \
'<ColorMapEntry color="#000096" quantity="62" />' + \
'<ColorMapEntry color="#000096" quantity="62" />' + \
'<ColorMapEntry color="#000096" quantity="61" />' + \
'<ColorMapEntry color="#000096" quantity="60" />' + \
'<ColorMapEntry color="#000096" quantity="59" />' + \
'<ColorMapEntry color="#000096" quantity="58" />' + \
'<ColorMapEntry color="#000096" quantity="57" />' + \
'<ColorMapEntry color="#000096" quantity="56" />' + \
'<ColorMapEntry color="#000096" quantity="55" />' + \
'<ColorMapEntry color="#000096" quantity="54" />' + \
'<ColorMapEntry color="#000096" quantity="53" />' + \
'<ColorMapEntry color="#000096" quantity="52" />' + \
'<ColorMapEntry color="#000096" quantity="52" />' + \
'<ColorMapEntry color="#000096" quantity="51" />' + \
'<ColorMapEntry color="#000096" quantity="50" />' + \
'<ColorMapEntry color="#000096" quantity="49" />' + \
'<ColorMapEntry color="#000096" quantity="48" />' + \
'<ColorMapEntry color="#000096" quantity="47" />' + \
'<ColorMapEntry color="#000096" quantity="46" />' + \
'<ColorMapEntry color="#000096" quantity="45" />' + \
'<ColorMapEntry color="#000096" quantity="44" />' + \
'<ColorMapEntry color="#000096" quantity="43" />' + \
'<ColorMapEntry color="#000096" quantity="42" />' + \
'<ColorMapEntry color="#000096" quantity="42" />' + \
'<ColorMapEntry color="#000096" quantity="41" />' + \
'<ColorMapEntry color="#000096" quantity="40" />' + \
'<ColorMapEntry color="#000096" quantity="39" />' + \
'<ColorMapEntry color="#000096" quantity="38" />' + \
'<ColorMapEntry color="#000096" quantity="37" />' + \
'<ColorMapEntry color="#000096" quantity="36" />' + \
'<ColorMapEntry color="#000096" quantity="35" />' + \
'<ColorMapEntry color="#000096" quantity="34" />' + \
'<ColorMapEntry color="#000096" quantity="33" />' + \
'<ColorMapEntry color="#000096" quantity="32" />' + \
'<ColorMapEntry color="#000096" quantity="31" />' + \
'<ColorMapEntry color="#000096" quantity="30" />' + \
'<ColorMapEntry color="#000096" quantity="29" />' + \
'<ColorMapEntry color="#000096" quantity="28" />' + \
'<ColorMapEntry color="#000096" quantity="27" />' + \
'<ColorMapEntry color="#000096" quantity="26" />' + \
'<ColorMapEntry color="#0000FF" quantity="25" />' + \
'<ColorMapEntry color="#0000FF" quantity="24" />' + \
'<ColorMapEntry color="#0000FF" quantity="23" />' + \
'<ColorMapEntry color="#0000FF" quantity="22" />' + \
'<ColorMapEntry color="#0000FF" quantity="22" />' + \
'<ColorMapEntry color="#0000FF" quantity="21" />' + \
'<ColorMapEntry color="#0000FF" quantity="20" />' + \
'<ColorMapEntry color="#0000FF" quantity="19" />' + \
'<ColorMapEntry color="#0000FF" quantity="18" />' + \
'<ColorMapEntry color="#0000FF" quantity="17" />' + \
'<ColorMapEntry color="#0000FF" quantity="16" />' + \
'<ColorMapEntry color="#0000FF" quantity="15" />' + \
'<ColorMapEntry color="#0000FF" quantity="14" />' + \
'<ColorMapEntry color="#0000FF" quantity="13" />' + \
'<ColorMapEntry color="#0000FF" quantity="12" />' + \
'<ColorMapEntry color="#0000FF" quantity="11" />' + \
'<ColorMapEntry color="#0070FF" quantity="10" />' + \
'<ColorMapEntry color="#0070FF" quantity="09" />' + \
'<ColorMapEntry color="#0070FF" quantity="08" />' + \
'<ColorMapEntry color="#0070FF" quantity="07" />' + \
'<ColorMapEntry color="#0070FF" quantity="06" />' + \
'<ColorMapEntry color="#6EBFFF" quantity="05" />' + \
'<ColorMapEntry color="#6EBFFF" quantity="04" />' + \
'<ColorMapEntry color="#6EBFFF" quantity="03" />' + \
'<ColorMapEntry color="#6EBFFF" quantity="02" />' + \
'<ColorMapEntry color="#6EBFFF" quantity="01" />' + \
'<ColorMapEntry color="#6EBFFF" quantity="00" />' + \
'<ColorMapEntry color="#6EBFFF" quantity="-01" />' + \
'<ColorMapEntry color="#6EBFFF" quantity="-02" />\n' + \
'<ColorMapEntry color="#6EBFFF" quantity="-03" />\n'
'<ColorMapEntry color="#D2D2D2" quantity="-04" />' + #R 210;G 210;B 210; lightgrey
'<ColorMapEntry color="#D2D2D2" quantity="-05" />' + \
'<ColorMapEntry color="#D2D2D2" quantity="-06" />' + \
'<ColorMapEntry color="#FFFFBE" quantity="-07" />' + #R 255;G 255;B 190; buffyellow
'<ColorMapEntry color="#FFFFBE" quantity="-08" />' + \
'<ColorMapEntry color="#FFFFBE" quantity="-09" />' + \
'<ColorMapEntry color="#FFFF00" quantity="-10" />' + #R 255;G 255;B 0; brightyellow
'<ColorMapEntry color="#FFFF00" quantity="-11" />' + \
'<ColorMapEntry color="#FFFF00" quantity="-12" />' + \
'<ColorMapEntry color="#FFD37F" quantity="-13" />' + #R 255;G 211;B 127; lightorange
'<ColorMapEntry color="#FFD37F" quantity="-14" />' + \
'<ColorMapEntry color="#FFD37F" quantity="-15" />' + \
'<ColorMapEntry color="#FFAA00" quantity="-16" />' + #R 255;G 170;B 0; redorange
'<ColorMapEntry color="#FFAA00" quantity="-17" />' + \
'<ColorMapEntry color="#FFAA00" quantity="-18" />' + \
'<ColorMapEntry color="#E64C00" quantity="-19" />' + #R 230;G 76;B 0; darkred
'<ColorMapEntry color="#E64C00" quantity="-20" />' + \
'<ColorMapEntry color="#E64C00" quantity="-21" />' + \
'<ColorMapEntry color="#A80000" quantity="-22" />' + #R 168;G 0;B 0; verydarkred
'<ColorMapEntry color="#A80000" quantity="-23" />' + \
'<ColorMapEntry color="#A80000" quantity="-24" />' + \
'<ColorMapEntry color="#A80000" quantity="-25" />' + \
'<ColorMapEntry color="#730000" quantity="-26" />' + #R 115;G 0;B 0; beet
'<ColorMapEntry color="#730000" quantity="-27" />' + \
'<ColorMapEntry color="#730000" quantity="-28" />' + \
'<ColorMapEntry color="#730000" quantity="-29" />' + \
'<ColorMapEntry color="#343434" quantity="-30" />' + #R 52;G 52;B 52; slate
'<ColorMapEntry color="#343434" quantity="-31" />' + \
'<ColorMapEntry color="#343434" quantity="-32" />' + \
'<ColorMapEntry color="#343434" quantity="-33" />' + \
'<ColorMapEntry color="#4C0073" quantity="-34" />' + #R 76;G 0;B 115; darkpurple
'<ColorMapEntry color="#4C0073" quantity="-35" />' + \
'<ColorMapEntry color="#4C0073" quantity="-36" />' + \
'<ColorMapEntry color="#4C0073" quantity="-37" />' + \
'<ColorMapEntry color="#8400A8" quantity="-38" />' + #R 132;G 0;B 168; purple
'<ColorMapEntry color="#8400A8" quantity="-39" />' + \
'<ColorMapEntry color="#8400A8" quantity="-40" />' + \
'<ColorMapEntry color="#8400A8" quantity="-41" />' + \
'<ColorMapEntry color="#8400A8" quantity="-42" />' + \
'<ColorMapEntry color="#8400A8" quantity="-43" />' + \
'<ColorMapEntry color="#8400A8" quantity="-44" />' + \
'<ColorMapEntry color="#8400A8" quantity="-45" />' + \
'<ColorMapEntry color="#8400A8" quantity="-46" />' + \
'<ColorMapEntry color="#8400A8" quantity="-47" />' + \
'<ColorMapEntry color="#8400A8" quantity="-48" />' + \
'<ColorMapEntry color="#8400A8" quantity="-49" />' + \
'<ColorMapEntry color="#8400A8" quantity="-50" />' + \
'<ColorMapEntry color="#8400A8" quantity="-51" />' + \
'<ColorMapEntry color="#8400A8" quantity="-52" />' + \
'<ColorMapEntry color="#8400A8" quantity="-53" />' + \
'<ColorMapEntry color="#8400A8" quantity="-54" />' + \
'<ColorMapEntry color="#8400A8" quantity="-55" />' + \
'<ColorMapEntry color="#8400A8" quantity="-56" />' + \
'<ColorMapEntry color="#8400A8" quantity="-57" />' + \
'<ColorMapEntry color="#8400A8" quantity="-58" />' + \
'<ColorMapEntry color="#8400A8" quantity="-59" />' + \
'<ColorMapEntry color="#8400A8" quantity="-60" />' + \
'<ColorMapEntry color="#8400A8" quantity="-61" />' + \
'<ColorMapEntry color="#8400A8" quantity="-62" />' + \
'<ColorMapEntry color="#8400A8" quantity="-63" />' + \
'<ColorMapEntry color="#8400A8" quantity="-64" />' + \
'<ColorMapEntry color="#8400A8" quantity="-65" />' + \
'<ColorMapEntry color="#8400A8" quantity="-66" />' + \
'<ColorMapEntry color="#8400A8" quantity="-67" />' + \
'<ColorMapEntry color="#8400A8" quantity="-68" />' + \
'<ColorMapEntry color="#8400A8" quantity="-69" />' + \
'<ColorMapEntry color="#8400A8" quantity="-70" />' + \
'<ColorMapEntry color="#8400A8" quantity="-71" />' + \
'<ColorMapEntry color="#8400A8" quantity="-72" />' + \
'<ColorMapEntry color="#8400A8" quantity="-73" />' + \
'<ColorMapEntry color="#8400A8" quantity="-75" />' + \
'<ColorMapEntry color="#8400A8" quantity="-76" />' + \
'<ColorMapEntry color="#8400A8" quantity="-77" />' + \
'<ColorMapEntry color="#8400A8" quantity="-78" />' + \
'<ColorMapEntry color="#8400A8" quantity="-79" />' + \
'<ColorMapEntry color="#8400A8" quantity="-80" />' + \
'<ColorMapEntry color="#8400A8" quantity="-81" />' + \
'<ColorMapEntry color="#8400A8" quantity="-82" />' + \
'<ColorMapEntry color="#8400A8" quantity="-83" />' + \
'<ColorMapEntry color="#8400A8" quantity="-84" />' + \
'<ColorMapEntry color="#8400A8" quantity="-85" />' + \
'<ColorMapEntry color="#8400A8" quantity="-86" />' + \
'<ColorMapEntry color="#8400A8" quantity="-87" />' + \
'<ColorMapEntry color="#8400A8" quantity="-88" />' + \
'<ColorMapEntry color="#8400A8" quantity="-89" />' + \
'<ColorMapEntry color="#8400A8" quantity="-90" />' + \
'<ColorMapEntry color="#8400A8" quantity="-91" />' + \
'<ColorMapEntry color="#8400A8" quantity="-92" />' + \
'<ColorMapEntry color="#8400A8" quantity="-93" />' + \
'<ColorMapEntry color="#8400A8" quantity="-94" />' + \
'<ColorMapEntry color="#8400A8" quantity="-95" />' + \
'<ColorMapEntry color="#8400A8" quantity="-96" />' + \
'<ColorMapEntry color="#8400A8" quantity="-97" />' + \
'<ColorMapEntry color="#8400A8" quantity="-98" />' + \
'<ColorMapEntry color="#8400A8" quantity="-99" />' + \
'<ColorMapEntry color="#8400A8" quantity="-93" />' + \
'<ColorMapEntry color="#8400A8" quantity="-94" />' + \
'<ColorMapEntry color="#8400A8" quantity="-95" />' + \
'<ColorMapEntry color="#8400A8" quantity="-96" />' + \
'<ColorMapEntry color="#8400A8" quantity="-97" />' + \
'<ColorMapEntry color="#8400A8" quantity="-98" />' + \
'<ColorMapEntry color="#8400A8" quantity="-99" />' + \
'<ColorMapEntry color="#8400A8" quantity="-100" />' + \
'<ColorMapEntry color="#8400A8" quantity="-101" />' + \
'<ColorMapEntry color="#8400A8" quantity="-102" />' + \
'<ColorMapEntry color="#8400A8" quantity="-103" />' + \
'<ColorMapEntry color="#8400A8" quantity="-104" />' + \
'<ColorMapEntry color="#8400A8" quantity="-105" />' + \
'<ColorMapEntry color="#8400A8" quantity="-106" />' + \
'<ColorMapEntry color="#8400A8" quantity="-107" />' + \
'<ColorMapEntry color="#8400A8" quantity="-108" />' + \
'<ColorMapEntry color="#8400A8" quantity="-109" />' + \
'<ColorMapEntry color="#8400A8" quantity="-110" />' + \
'<ColorMapEntry color="#8400A8" quantity="-111" />' + \
'<ColorMapEntry color="#8400A8" quantity="-112" />' + \
'<ColorMapEntry color="#8400A8" quantity="-113" />' + \
'<ColorMapEntry color="#8400A8" quantity="-114" />' + \
'<ColorMapEntry color="#8400A8" quantity="-115" />' + \
'<ColorMapEntry color="#8400A8" quantity="-116" />' + \
'<ColorMapEntry color="#8400A8" quantity="-117" />' + \
'<ColorMapEntry color="#8400A8" quantity="-118" />' + \
'<ColorMapEntry color="#8400A8" quantity="-119" />' + \
'<ColorMapEntry color="#8400A8" quantity="-120" />' + \
'<ColorMapEntry color="#8400A8" quantity="-121" />' + \
'<ColorMapEntry color="#8400A8" quantity="-122" />' + \
'<ColorMapEntry color="#8400A8" quantity="-123" />' + \
'<ColorMapEntry color="#8400A8" quantity="-124" />' + \
'<ColorMapEntry color="#8400A8" quantity="-125" />' + \
'<ColorMapEntry color="#8400A8" quantity="-126" />' + \
'<ColorMapEntry color="#8400A8" quantity="-127" />' + \
'</ColorMap>' + \
'</RasterSymbolizer>')
# '<ColorMapEntry color="#ECECEC" quantity="-01" />\n' +  # optional light grey start at -1
# '<ColorMapEntry color="#ECECEC" quantity="-02" />' + \
# '<ColorMapEntry color="#ECECEC" quantity="-03" />' + \

In [49]:
####  LEGEND GRAPHICS  ########################/
# https:#drive.google.com/drive/folders/1GhX4BUNjrA6FWehs1aXi1Gl_fAqf_c5f?usp=sharing
####################################/

###  grey hillshade
dataset = ee.Image('USGS/3DEP/10m')
elevation = dataset.select('elevation')
hillshade = ee.Terrain.hillshade(elevation)
datasetVis = {
  'min': 0.0,
  'max': 255.0,
  'gamma': 0.50, 'opacity':0.50
}
Ten_m_clip = hillshade.clip(geometry)
m.addLayer(Ten_m_clip, datasetVis, 'hillshade', False)

In [24]:
##################################
#  NLCD https:#developers.google.com/earth-engine/datasets/catalog/USGS_NLCD_RELEASES_2019_REL_NLCD

# Import the NLCD collection.
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')

# The collection contains images for multiple years and regions in the USA.
import pprint
# print(dataset.getInfo())
image_ids = dataset.aggregate_array('system:index').getInfo()
# print('Images:', image_ids)

# Filter the collection to the 2016 product.
# print('Products:', dataset.aggregate_array('system:index'))

# Filter the collection to the 2016 product.
nlcd2019 = dataset.filter(ee.Filter.eq('system:index', '2019')).first()

# Each product has multiple bands for describing aspects of land cover.
print('Bands:', nlcd2019.bandNames().getInfo())

# Select the land cover band.
landcover = nlcd2019.select('landcover')

# Display land cover on the map.
m = geemap.Map()
m.setCenter(-95, 38, 5)
m.addLayer(landcover, None, 'Landcover')
# m

#############################
# NLCD Classes
#############################

# 11 Open water
# 12 Perennial ice/snow
# 21 Developed, open space
# 22 Developed, low intensity
# 23 Developed, medium intensity
# 24 Developed high intensity
# 31 Barren land (rock/sand/clay)
# 41 Deciduous forest
# 42 Evergreen forest
# 43 Mixed forest
# 51 Dwarf scrub, Alaska only
# 52 Shrub/scrub
# 71 Grassland/herbaceous
# 72 Sedge/herbaceous
# 73 Lichens, Alaska only
# 74 Moss, Alaska only
# 81 Pasture/hay
# 82 Cultivated crops
# 90 Woody wetlands
# 95 Emergent herbaceous wetlands

Bands: ['landcover', 'impervious', 'impervious_descriptor']


In [51]:
##################################
# bring in 2019 NLCD - define 4 forest classes = forest2019
#################################/

dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')
nlcd2019 = dataset.filter(ee.Filter.eq('system:index', '2019')).first()
nlcd_2019_landcover_img = nlcd2019.select('landcover')

forest2019 =  nlcd_2019_landcover_img.eq(41).Or(nlcd_2019_landcover_img.eq(42)).Or(nlcd_2019_landcover_img.eq(43)).Or(nlcd_2019_landcover_img.eq(90))
pineforest2019 =  nlcd_2019_landcover_img.eq(42).Or(nlcd_2019_landcover_img.eq(43))
hwdforest2019 =  nlcd_2019_landcover_img.eq(41).Or(nlcd_2019_landcover_img.eq(90))

# all 4-forest classes
nlcd_wild = nlcd_2019_landcover_img.eq(41).\
          Or(nlcd_2019_landcover_img.eq(42)).\
          Or(nlcd_2019_landcover_img.eq(43)).\
          Or(nlcd_2019_landcover_img.eq(90))
nlcd_wild_mask = nlcd_wild.mask(nlcd_wild)
mST = mean_ST2_x100sint8_binned
mST_mask = mean_ST2_x100sint8_binned.mask(mean_ST2_x100sint8_binned)
# evgrn_range_x100sint8_mask = evgrn_range_x100sint8.mask(evgrn_range_x100sint8)
# dec_range_x100sint8_mask = dec_range_x100sint8.mask(dec_range_x100sint8)

# eg_2masks = nlcd_wild.mask(nlcd_wild).mask(evgrn_range_x100sint8.mask(evgrn_range_x100sint8))
# eg_3masks = eg_2masks.mask(mST)
# dec_2masks = nlcd_wild.mask((nlcd_wild).mask(dec_range_x100sint8.mask(dec_range_x100sint8)))
# dec_3masks = dec_2masks.mask(mST)

# m.addLayer(slope.mask(forest2019).clip(geometry), {min: 0, max: 60, palette: [
#       '3ae237', 'b5e22e', 'd6e21f', 'fff705', 'ffd611', 'ffb613', 'ff8b13',
#     'ff6e08', 'ff500d', 'ff0000', 'de0101', 'c21301', '0602ff', '235cb1',
#     '307ef3', '269db1', '30c8e2', '32d3ef', '3be285', '3ff38f', '86e26f'
#   ],
# }, 'forested slopes ', False)

# m.addLayer(eg_3masks, {}, 'eg_3masks', False)
# m.addLayer(dec_3masks, {}, 'dec_3masks', False)
# m.addLayer(absNDVIc_x100sint8.sldStyle(sld_intervals).mask(eg_3masks), {}, 'abs eg_3masks', False)
# m.addLayer(absNDVIc_x100sint8.sldStyle(sld_intervals).mask(dec_3masks), {}, 'abs dec_3masks', False)

# masking all 4-forest classes with meanST
nlcd4_mST_mask = nlcd_wild.mask(nlcd_wild).mask(mean_ST2_x100sint8_binned.mask(mean_ST2_x100sint8_binned))
# m.addLayer(nlcd_wild_mask, {}, 'nlcd_wild', False)
# m.addLayer(mST_mask, {}, 'mST_mask', False)
# m.addLayer(nlcd4_mST_mask, {}, 'nlcd4_mST_mask', False)

In [52]:
###################
##  ABSOLUTE CHANGE
###################
m.addLayer(absNDVIc_x100sint8.sldStyle(sld_intervals).clip(geometry), {}, 'all-lands ndvi change', False)

#m.addLayer(absNDVIc_x100sint8.sldStyle(sld_intervals).mask(forest2019).clip(geometry), {}, 'forest ABS change', False)
m.addLayer(absNDVIc_x100sint8.sldStyle(sld_intervals).mask(nlcd4_mST_mask).clip(geometry), {}, 'forest ndvi change', False)

##  Export forest-masked, photo-like RGB raster, pre-colorized, not editable on desktop
#Export.image.toDrive(**{'image': absNDVIc_x100sint8.sldStyle(sld_intervals).mask(nlcd4_mST_mask).clip(geometry), 'description': 'S2TOA_2bins_severeANDmoderate_1yrNDVIchange_092122_RGB_pre052121_072121_post072122_092122_v030923', 'region':geometry,'scale':10, 'maxPixels':1e13})

##  Export forest-masked, continuous value raster, requires color map, editable on desktop, threshold/isolate ndvi change values
#Export.image.toDrive(**{image: absNDVIc_x100sint8.sldStyle(sld_intervals).mask(nlcd4_mST_mask).clip(geometry), description: 'S2SR_Harvests_HarrisCo_GA_1yrNDVIchange_092122_sINT8bit_pre072121_092121_post072122_092122_v030723',region:geometry,'scale':10, 'maxPixels':1e13})
#    filename suggestion: sensor_subject_location_state_temporal_productformat_predates_postdates_versiondate

# mask by Seasonal Thresholding (ST)
#m.addLayer(absNDVIc_x100sint8.sldStyle(sld_intervals).mask(dec_range_x100sint8).clip(geometry), {}, 'dec forest departure by ST', False)
#m.addLayer(absNDVIc_x100sint8.sldStyle(sld_intervals).mask(evgrn_range_x100sint8).clip(geometry), {}, 'evgrn-mixed forest departure by ST', False)

# Export hybrid-forest-masked, photo-like RGB composite, pre-colorized
#Export.image.toDrive(**{image: absNDVIc_x100sint8.sldStyle(sld_intervals).mask(decforest_mask).clip(geometry), description: 'S2TOA_H_Sally_091620_DecForest20162006_dNDVI_RGB_pre071520_091520_post091720_100420_v100620',region:geometry,'scale':10, 'maxPixels':1e13})
#Export.image.toDrive(**{image: absNDVIc_x100sint8.sldStyle(sld_intervals).mask(evgrnforest_mask).clip(geometry), description: 'S2TOA_H_Sally_091620_EvgrnForest20162006_dNDVI_RGB_pre071520_091520_post091720_100420_v100620',region:geometry,'scale':10, 'maxPixels':1e13})
#    filename suggestion -                               sensor-state-event-evdate-product-format-preimage-postimage-versiondate

In [53]:
################################
###  MEDIUM-SEVERITY NDVI-departure value 'Thresholding of DN' (TDN) -
# can be applied to non-masked or masked, DN or RGB products
################################

forchange2 = absNDVIc_x100sint8.mask(nlcd4_mST_mask).clip(geometry) # DEFAULT, reports both evergreen and deciduous harvest combined
# forchange = absNDVIc_x100sint8.mask(eg_3masks).clip(geometry); # for evergreen harvest output
# forchange = absNDVIc_x100sint8.mask(dec_3masks).clip(geometry); # for deciduous harvest output, to make active remove the #, and comment-out line 876 with # at beginning of line

#forchange_tdn = forchange.mask(forchange.select(['NDVI']).lte(-20))
forchange_tdn2 = forchange2.mask((forchange2.select(['NDVI']).lte(-7)).And(forchange2.select(['NDVI']).gte(-19)))

tholdmask2 = forchange_tdn2.select(['NDVI']).lte(-7).And(forchange_tdn2.select(['NDVI']).gte(-19))
#print(tholdmask,'tholdmask')
#m.addLayer(tholdmask2.clip(geometry), {}, 'Forest Thold mask', False)

# # Compute the number of pixels in each patch.
# #patchsize = tholdmask.connectedPixelCount(100, False)
patchsize2 = tholdmask2.connectedPixelCount(200, False)  # default 100
# #m.addLayer(patchsize, {palette: ['000000', 'FFFFFF']}, 'patch size qualifier', False)
# mmu = 50
# # mmu2 = mmu+5
# # print(mmu2,'mmu2')

# patchsize = tholdmask.connectedPixelCount(mmu, False)
#m.addLayer(forchange_tdn.sldStyle(sld_intervals), {}, 'Forest Thold change RGB', False)
# m.addLayer(forchange_tdn.mask().clip(geometry), {}, 'Forest Thold change')

# Get rid of any patches that are less than 36 pixels.
patchsizeLarge2 = patchsize2.gte(200)  # 40pix = 1ac, 50pix = 0.5 ha
patchsizeLarge2 = patchsizeLarge2.updateMask(patchsizeLarge2)
# m.addLayer(patchsizeLarge, {palette: ['000000', 'FFFFFF']}, 'patch size large binary', False)
# print(patchsizeLarge,'patchsizeLarge')

In [54]:
################################
###  Polygonize thresholding DN product
################################

zones2 = forchange_tdn2.lte(-7).And(forchange_tdn2.select(['NDVI']).gte(-19)).mask(patchsizeLarge2)
zones2 = zones2.updateMask(zones2.neq(0))
#print(zones,'zones')

# Convert the zones of the thresholded change to vectors
vectors2 = zones2.addBands((forchange_tdn2)).reduceToVectors(**{
  'geometry': geometry,
  'crs': forchange_tdn2.projection(),
  'scale': 10,
  'geometryType': 'polygon',
  'eightConnected': False,
  'labelProperty': 'zone',
  'reducer': ee.Reducer.mean(),
  'maxPixels':1e13
})

#m.addLayer(vectors2, {palette: 'FF2400'}, 'moderate NDVI change (black filled poly)', False)

#  Add perim outlines to the map as img
fp2 = ee.FeatureCollection(vectors2)
empty2 = ee.Image().byte()
outline2 = empty.paint(**{
  'featureCollection': fp2,
  'color': 2,
  'width': 1})
m.addLayer(outline2.clip(geometry), {'palette': '3DED97'}, 'moderate NDVI change (green outline poly)', False)
#m.addLayer(vectors, {}, 'pslrg polyon vectors as fc')

#Export.table.toDrive(**{collection: vectors, description: 'polys_large_patch_pine_thold', fileFormat: 'SHP'})

# export polygon shapefile to google drive
# Export.table.toDrive(**{'collection': vectors2, 'description': 'moderate_NDVI_change_polygons_SHAPEFILE', 'fileFormat': 'SHP'})
# Export.table.toDrive(**{collection: vectors, description: 'al_autauga_2022_eg_harvest_3masks2_n20dNDVI_pt5ha_polys', fileFormat: 'SHP'})
# Export.table.toDrive(**{collection: vectors, description: 'gfcD5_2022_dec_harvest_3masks2_n20dNDVI_pt5ha_polys', fileFormat: 'SHP'})

# Compute centroids of each polygon

def func_kak(f):
  return f.centroid(**{'maxError':1})
centroids2 = vectors2.map(func_kak)

m.addLayer(centroids2, {'color': 'green'}, 'moderate NDVI decline (green points)', False)

#print(centroids,'centroids')

# Transform coordinates into properties in the table.

def func_sta (feature):
  # Get geometry
  coordinates2 = feature.geometry() \
                          .transform('epsg:4326') \
                          .coordinates()
  # Get both entries of coordinates and set them as new properties
  resul2 = feature.set('long', coordinates2.get(0),
                     'lat', coordinates2.get(1))
  # Remove geometry
  return resul2.setGeometry(None)

centroidsExport2 = centroids2.map(func_sta)

#Export.table.toDrive(**{'collection': centroids2, 'description': 'moderate_NDVI_change_centroids_SHAPEFILE', 'fileFormat': 'SHP'})
#Export.table.toDrive(**{'collection': centroidsExport2, 'selectors': ['lat',  'long'], 'description': 'moderate_NDVI_change_centroids_TEXTFILE', 'fileFormat': 'CSV'})

In [55]:
################################
###  HIGH-SEVERITY NDVI-departure value 'Thresholding of DN' (TDN) - can be applied to non-masked or masked, DN or RGB products
################################

forchange = absNDVIc_x100sint8.mask(nlcd4_mST_mask).clip(geometry); # DEFAULT, reports both evergreen and deciduous harvest combined
# forchange = absNDVIc_x100sint8.mask(eg_3masks).clip(geometry); # for evergreen harvest output
# forchange = absNDVIc_x100sint8.mask(dec_3masks).clip(geometry); # for deciduous harvest output, to make active remove the #, and comment-out line 876 with # at beginning of line

forchange_tdn = forchange.mask(forchange.select(['NDVI']).lte(-20))
# # forchange_tdn = forchange.mask((forchange.select(['NDVI']).lt(-3)).And(forchange.select(['NDVI']).gt(-128)))

tholdmask = forchange_tdn.select(['NDVI']).lte(-20)
#print(tholdmask,'tholdmask')
#m.addLayer(tholdmask.clip(geometry), {}, 'Forest Thold mask', False)

# # Compute the number of pixels in each patch.
# #patchsize = tholdmask.connectedPixelCount(100, False)
patchsize = tholdmask.connectedPixelCount(100, False);  # default 100
# #m.addLayer(patchsize, {palette: ['000000', 'FFFFFF']}, 'patch size qualifier', False)
# mmu = 50
# # mmu2 = mmu+5
# # print(mmu2,'mmu2')

# patchsize = tholdmask.connectedPixelCount(mmu, False)
#m.addLayer(forchange_tdn.sldStyle(sld_intervals), {}, 'Forest Thold change RGB', False)
# m.addLayer(forchange_tdn.mask().clip(geometry), {}, 'Forest Thold change')

# Get rid of any patches that are less than 36 pixels.
patchsizeLarge = patchsize.gte(100);  # 40pix = 1ac, 50pix = 0.5 ha
patchsizeLarge = patchsizeLarge.updateMask(patchsizeLarge)
# m.addLayer(patchsizeLarge, {palette: ['000000', 'FFFFFF']}, 'patch size large binary', False)
# print(patchsizeLarge,'patchsizeLarge')

In [56]:
################################
###  Polygonize thresholding DN product
################################

zones = forchange_tdn.lte(-20).mask(patchsizeLarge)
zones = zones.updateMask(zones.neq(0))
#print(zones,'zones')

# Convert the zones of the thresholded change to vectors
vectors = zones.addBands((forchange_tdn)).reduceToVectors(**{
  'geometry': geometry,
  'crs': forchange_tdn.projection(),
  'scale': 10,
  'geometryType': 'polygon',
  'eightConnected': False,
  'labelProperty': 'zone',
  'reducer': ee.Reducer.mean(),
  'maxPixels':1e13
})

#m.addLayer(vectors, {palette: 'FF2400'}, 'severe NDVI change (clear cut?) (black filled poly)', False)

#  Add perim outlines to the map as img
fp = ee.FeatureCollection(vectors)
empty = ee.Image().byte()
outline = empty.paint(**{
  'featureCollection': fp,
  'color': 2,
  'width': 1})
#m.addLayer(outline.clip(geometry), {palette: 'FF2400'}, 'severe NDVI change (red outline poly)')
m.addLayer(outline.clip(geometry), {'palette': 'FF2400'}, 'severe NDVI change (red outline poly)', False)
#m.addLayer(vectors, {}, 'pslrg polyon vectors as fc')

#Export.table.toDrive(**{collection: vectors, description: 'polys_large_patch_pine_thold', fileFormat: 'SHP'})

# export polygon shapefile to google drive
# Export.table.toDrive(**{'collection': vectors, 'description': 'severe_NDVI_change_polygons_SHAPEFILE', 'fileFormat': 'SHP'})
# Export.table.toDrive(**{collection: vectors, description: 'al_autauga_2022_eg_harvest_3masks2_n20dNDVI_pt5ha_polys', fileFormat: 'SHP'})
# Export.table.toDrive(**{collection: vectors, description: 'gfcD5_2022_dec_harvest_3masks2_n20dNDVI_pt5ha_polys', fileFormat: 'SHP'})

# Compute centroids of each polygon

def func_jhk(f):
  return f.centroid(**{'maxError':1})
centroids = vectors.map(func_jhk)

m.addLayer(centroids, {'color': 'red'}, 'severe NDVI change (red points)', False)

#print(centroids,'centroids')

# Transform coordinates into properties in the table.

def func_cey (feature):
  # Get geometry
  coordinates = feature.geometry() \
                          .transform('epsg:4326') \
                          .coordinates()
  # Get both entries of coordinates and set them as new properties
  resul = feature.set('long', coordinates.get(0),
                    'lat', coordinates.get(1))
  # Remove geometry
  return resul.setGeometry(None)

centroidsExport = centroids.map(func_cey)

# Export.table.toDrive(**{'collection': centroids, 'description': 'severe_NDVI_change_centroids_SHAPEFILE', 'fileFormat': 'SHP'})
# Export.table.toDrive(**{'collection': centroidsExport, 'selectors': ['lat',  'long'], 'description': 'severe_NDVI_change_centroids_TEXTFILE', 'fileFormat': 'CSV'})


In [57]:
######################################################
#  streams raster, change the state designation at line 1040 to 'your' state's lowercase-abbreviation
######################################################

streams = ee.Image('users/timberharvestmap/nhd_ga')
streams_clip = streams.clip(geometry)
m.addLayer(streams_clip, {'palette': ['0700d6']}, 'streams, blue (USGS NHD-HR)', False)
m.addLayer(streams_clip, {'palette': ['ffff00']}, 'streams, yellow (for dark backgrounds)', False)


In [58]:
################################

###  Add perim outlines to the map

# msboe = ee.FeatureCollection('users/srs4854gee/Christie/MS/MS_BOE_033020')
# empty = ee.Image().byte()
# msboe_outline = empty.paint(**{
#   featureCollection: msboe,
#   color: 1,
#   width: 2})
# #m.addLayer(msboe_outline.clip(geometry), {palette: '000000'}, 'ms boe', False)
# m.addLayer(msboe_outline, {palette: '000000'}, 'ms boe', False)

###  Display a point shapefile, requires a point shapefile be imported to assets and script

#m.addLayer(table,{palette: 'ff0000'},'spb points')
#fcVis = table.draw(**{color: 'ECD803', pointRadius: 10, strokeWidth: 3})
#m.addLayer(fcVis, False)

###  Boundaries US States and Counties

dataset = ee.FeatureCollection('users/timberharvestmap/gen_co20m_minatt_east')

visParams = {
  'palette': ['purple', 'blue', 'green', 'yellow', 'orange', 'red'],
  'min': 0,
  'max': 50,
  'opacity': 0.8,
}
stateDataset = ee.FeatureCollection('TIGER/2016/States')
# Turn the strings into numbers

def func_ziv (f):
  return f.set('STATEFP', ee.Number.parse(f.get('STATEFP')))

dataset = dataset.map(func_ziv)

image = ee.Image().float().paint(dataset, 'STATEFP')
countyOutlines = ee.Image().float().paint(**{
  'featureCollection': dataset,
  'color': 'black',
  'width': 2
})
stateOutlines = ee.Image().float().paint(**{
  'featureCollection': stateDataset,
  'color': 'black',
  'width': 2.5
})

#m.addLayer(countyOutlines, {}, 'county outlines', False)
m.addLayer(countyOutlines, {}, 'county outlines')
m.addLayer(stateOutlines, {}, 'state outlines', False)

In [59]:
#############################
##  Add FS Surface Owned outlines to the map
#############################

fsowned = ee.FeatureCollection('users/timberharvestmap/fs_surface_own_042622_east')
empty = ee.Image().byte()
outline5 = empty.paint(**{
  'featureCollection': fsowned,
  'color': 1,
  'width': 1})

In [60]:
#############################
##  Add FS Proclaimed Admin outlines to the map
#############################  users/srs4854gee/el_yunque_nf_sown

fsproc = ee.FeatureCollection('users/timberharvestmap/fs_proclaimed_042622_east')
#fsproc = ee.FeatureCollection('users/srs4854gee/el_yunque_nf_sown')
empty = ee.Image().byte()
outline = empty.paint(**{
  'featureCollection': fsproc,
  'color': 1,
  'width': 2})

m.addLayer(outline, {'palette': '66ff00'}, 'usfs proclaimed boundary')
m.addLayer(outline5, {'palette': '66ff00'}, 'usfs surface owned boundary', False)

#############################
m

Map(center=[32.73709112028248, -84.90534048722103], controls=(WidgetControl(options=['position', 'transparent_…